In [22]:
import pandas as pd
import os
import numpy as np
import requests
import lxml
from bs4 import BeautifulSoup
import re

In [23]:
#pull the contents of the website
r=requests.get('https://data.nsw.gov.au/data/dataset/fuel-check')

In [24]:
r.status_code #200 means download was successful

200

In [25]:
#tried lxml, it failed
soup=BeautifulSoup(r.content,"html.parser")
#transfer content into beautifulsoup, so i can filter what i want

In [26]:
#https://stackoverflow.com/questions/35416575/get-the-href-text-of-a-link-that-has-a-certain-class-attribute-using-beautifulso
a=soup.find_all('a', {'class':"resource-url-analytics",'href':True})
#looked at the website, and knew exactly where my data was...

In [27]:
#this loops through and appends all the files to be downloaded
b=[]
for i in a:
    b.append(i['href'])

In [28]:
#strips out url that does not end with .xlsx
b=[j for j in b if j.endswith('.xlsx')]

In [29]:
z=pd.Series(data=b) #create a Series

In [30]:
#redundancy 
z.to_csv('/home/sam/Github Folder/FuelCheck/fuelcheckurllist.csv')

In [31]:
#create a dictionary of months and abbreviations, which comes in handy
#in the next cell... if there are any abbreviations of the month,
#they will be replaced by the full month name.
m_dict={'jan':'january','feb':'february', 'mar':'march','apr':'april',\
        'jun':'jun','jul':'july','aug':'august','sep':'september',\
         'oct':'october','nov':'november','dec':'december'}

#declare an empty list variable
url_m_y=[]
for j in b:
#split the string based on '/', strip the end of the '.xlsx'
#and pull out the last string/word at the end using index notation [-1]
    m_y_extract=j.split('/')[-1].rstrip('.xlsx')
#some have '-' present, others do not. So I have to treat the extraction
#differently.
#i used [-7:], since the year is four digits and abbreviation is usually
#three. so the '-' be before the year. 
#thinking about it, i could have made it [-6:]...yeah yeah.
    if '-' in m_y_extract[-7:]:
#i first split by '-', then slice it to get the month and year,
#join it with '-' and add '.csv' at the end
        m_y='-'.join(m_y_extract.split('-')[-2:])+'.csv'
    if '-' not in m_y_extract[-7:]:
#'-' not present, so a different procedure
#extract the text before the year (the abbreviations are three digits)
        p=m_y_extract[-7:][:3]
#extract the year
        q=m_y_extract[-7:][3:]
#add them back with the '.csv'
        m_y=p+'-'+q+'.csv'
#now the dictionary comes into play
#apparently abbreviations are present, and I need the full names of the
#months for further work down.
#I split 'm_y' into month and year
    m=m_y.split('-')[0]
    y=m_y.split('-')[-1]
#I check if the abbreviation is present, and replace it with the full
#month name.
    if m in m_dict.keys():
        m_y=m_dict[m]+'-'+y
    url_m_y.append(m_y)
        

#create a dataframe of the url and month_year
#comes in handy during the filter phase to determine which datasets 
#are already present, and which to download
df_url=pd.DataFrame({'urllist':b,'url_month_year':url_m_y})

#this pulls out the files in a directory...
#the s.lower() converts to lowercase, so the work in the next cell
#wont have any case (capital vs small letters) issues
fuelcheck_filelist=[s.lower() for s in os.listdir('/home/sam/Github Folder/FuelCheck')]

#using set notation, i find out which files have not been downloaded
fuel_list_to_download=set(url_m_y)-set(fuelcheck_filelist)
fuel_list_to_download=list(fuel_list_to_download)
fuel_list_to_download

#https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
#i then pass the filter above into the dataframe created
url_listt=df_url[df_url['url_month_year'].isin(fuel_list_to_download)]['urllist'].tolist()

#now i can download the specific files, and avoid downloading files
#that already exist in my local folder.
#i also convert to csv as it is much faster to process than .xlsx
for i in url_listt:
    file_download=pd.read_excel(i)
    if file_download.columns[6] !='PriceUpdatedDate':
        file_download.columns=file_download.iloc[0]
        file_download=file_download.drop(index=0)
        file_download=file_download.reset_index(drop=True)
    file_name=file_download['PriceUpdatedDate'][0].strftime("%B-%Y")
    file_download.to_csv(('/home/Github Folder/FuelCheck/{}.csv').format(file_name))
    